In [1]:
import requests
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

api_key = os.environ.get('API_KEY')
api_url = os.environ.get('API_URL')

In [3]:
def send_search_request(query_str):
    request_body = {
        "query": query_str,
        "page": 1
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.post(api_url + "/search", json=request_body, headers=headers)
    return response.json()

# results = send_search_request('Software engineer programmer starts to realize the magic system is akin to programming')
# for result in results:
#     index = result['DocEmbedding']["index"]
#     story_id = result['DocEmbedding']["story_id"]
#     link = f'https://www.royalroad.com/fiction/{story_id}'
#     print(link, index)

In [4]:
def send_search_request_document_group(query_str, group_size):
    request_body = {
        "doc_group_size": group_size,
        "query": query_str,
        "page": 1
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.post(api_url + "/search", json=request_body, headers=headers)
    return response.json()

In [5]:
def create_document_group(group_size):
    request_body = {
        "doc_group_size": group_size
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.post(api_url + "/document_group", json=request_body, headers=headers)
    return response.status_code

In [6]:
# create_document_group(50)

In [7]:
def index_document_group(story_id, group_size):
    request_body = {
        "story_id": story_id,
        "doc_group_size": group_size
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.put(api_url + "/document_group", json=request_body, headers=headers)
    return response.status_code

In [11]:
index_document_group(11209, 50)

204

In [38]:
# results = send_search_request_document_group('Software engineer programmer starts to realize the magic system is akin to programming', 50)

# for result in results:
#     story_id = result['DocGroupEmbedding']["story_id"]
#     index = result['DocGroupEmbedding']["index"]
#     link = f'https://www.royalroad.com/fiction/{story_id}'
#     print(link, index)

https://www.royalroad.com/fiction/13143 0
https://www.royalroad.com/fiction/26163 0
https://www.royalroad.com/fiction/28995 0
https://www.royalroad.com/fiction/24 0
https://www.royalroad.com/fiction/20177 0
https://www.royalroad.com/fiction/23074 0
https://www.royalroad.com/fiction/16878 0
